In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import sys
import statsmodels.api as sm

from sklearn.pipeline import Pipeline

# used for train/test splits
from sklearn.cross_validation import train_test_split

# used to impute mean for data
from sklearn.preprocessing import Imputer

# normalize the data
from sklearn import preprocessing

# logistic regression is our model of choice
from sklearn.linear_model import LogisticRegression

# used to create confusion matrix
from sklearn.metrics import confusion_matrix

from sklearn.cross_validation import cross_val_score

# used to calculate AUROC/accuracy
from sklearn import metrics

# for calibration curve of severity scores
from sklearn.calibration import calibration_curve


# default colours for prettier plots
col = [[0.9047, 0.1918, 0.1988],
    [0.2941, 0.5447, 0.7494],
    [0.3718, 0.7176, 0.3612],
    [1.0000, 0.5482, 0.1000],
    [0.4550, 0.4946, 0.4722],
    [0.6859, 0.4035, 0.2412],
    [0.9718, 0.5553, 0.7741],
    [0.5313, 0.3359, 0.6523]];
marker = ['v','o','d','^','s','o','+']
ls = ['-','-','-','-','-','s','--','--']
%matplotlib inline

In [ ]:
# create a database connection

# below config used on pc70
sqluser = 'alistairewj'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()

# Antibiotics usage

We can extract antibiotic usage from:

* PRESCRIPTIONS table
* INPUTEVENTS_MV and INPUTEVENTS_CV tables

First let's define antibiotics using the D_ITEMS table.

In [ ]:
# antibiotics
cur.execute('SET search_path to ' + schema_name)
# define antibiotics query in a new string
# this allows us to reuse it later
query_ab = \
"""
with ab as
(
select 
  di.*, linksto
  , case
    when lower(label) like '%' || lower('adoxa') || '%' then 1
    when lower(label) like '%' || lower('ala-tet') || '%' then 1
    when lower(label) like '%' || lower('alodox') || '%' then 1
    when lower(label) like '%' || lower('amikacin') || '%' then 1
    when lower(label) like '%' || lower('amikin') || '%' then 1
    when lower(label) like '%' || lower('amoxicillin') || '%' then 1
    when lower(label) like '%' || lower('amoxicillin%clavulanate') || '%' then 1
    when lower(label) like '%' || lower('clavulanate') || '%' then 1
    when lower(label) like '%' || lower('ampicillin') || '%' then 1
    when lower(label) like '%' || lower('augmentin') || '%' then 1
    when lower(label) like '%' || lower('avelox') || '%' then 1
    when lower(label) like '%' || lower('avidoxy') || '%' then 1
    when lower(label) like '%' || lower('azactam') || '%' then 1
    when lower(label) like '%' || lower('azithromycin') || '%' then 1
    when lower(label) like '%' || lower('aztreonam') || '%' then 1
    when lower(label) like '%' || lower('axetil') || '%' then 1
    when lower(label) like '%' || lower('bactocill') || '%' then 1
    when lower(label) like '%' || lower('bactrim') || '%' then 1
    when lower(label) like '%' || lower('bethkis') || '%' then 1
    when lower(label) like '%' || lower('biaxin') || '%' then 1
    when lower(label) like '%' || lower('bicillin l-a') || '%' then 1
    when lower(label) like '%' || lower('cayston') || '%' then 1
    when lower(label) like '%' || lower('cefazolin') || '%' then 1
    when lower(label) like '%' || lower('cedax') || '%' then 1
    when lower(label) like '%' || lower('cefoxitin') || '%' then 1
    when lower(label) like '%' || lower('ceftazidime') || '%' then 1
    when lower(label) like '%' || lower('cefaclor') || '%' then 1
    when lower(label) like '%' || lower('cefadroxil') || '%' then 1
    when lower(label) like '%' || lower('cefdinir') || '%' then 1
    when lower(label) like '%' || lower('cefditoren') || '%' then 1
    when lower(label) like '%' || lower('cefepime') || '%' then 1
    when lower(label) like '%' || lower('cefotetan') || '%' then 1
    when lower(label) like '%' || lower('cefotaxime') || '%' then 1
    when lower(label) like '%' || lower('cefpodoxime') || '%' then 1
    when lower(label) like '%' || lower('cefprozil') || '%' then 1
    when lower(label) like '%' || lower('ceftibuten') || '%' then 1
    when lower(label) like '%' || lower('ceftin') || '%' then 1
    when lower(label) like '%' || lower('cefuroxime ') || '%' then 1
    when lower(label) like '%' || lower('cefuroxime') || '%' then 1
    when lower(label) like '%' || lower('cephalexin') || '%' then 1
    when lower(label) like '%' || lower('chloramphenicol') || '%' then 1
    when lower(label) like '%' || lower('cipro') || '%' then 1
    when lower(label) like '%' || lower('ciprofloxacin') || '%' then 1
    when lower(label) like '%' || lower('claforan') || '%' then 1
    when lower(label) like '%' || lower('clarithromycin') || '%' then 1
    when lower(label) like '%' || lower('cleocin') || '%' then 1
    when lower(label) like '%' || lower('clindamycin') || '%' then 1
    when lower(label) like '%' || lower('cubicin') || '%' then 1
    when lower(label) like '%' || lower('dicloxacillin') || '%' then 1
    when lower(label) like '%' || lower('doryx') || '%' then 1
    when lower(label) like '%' || lower('doxycycline') || '%' then 1
    when lower(label) like '%' || lower('duricef') || '%' then 1
    when lower(label) like '%' || lower('dynacin') || '%' then 1
    when lower(label) like '%' || lower('ery-tab') || '%' then 1
    when lower(label) like '%' || lower('eryped') || '%' then 1
    when lower(label) like '%' || lower('eryc') || '%' then 1
    when lower(label) like '%' || lower('erythrocin') || '%' then 1
    when lower(label) like '%' || lower('erythromycin') || '%' then 1
    when lower(label) like '%' || lower('factive') || '%' then 1
    when lower(label) like '%' || lower('flagyl') || '%' then 1
    when lower(label) like '%' || lower('fortaz') || '%' then 1
    when lower(label) like '%' || lower('furadantin') || '%' then 1
    when lower(label) like '%' || lower('garamycin') || '%' then 1
    when lower(label) like '%' || lower('gentamicin') || '%' then 1
    when lower(label) like '%' || lower('kanamycin') || '%' then 1
    when lower(label) like '%' || lower('keflex') || '%' then 1
    when lower(label) like '%' || lower('ketek') || '%' then 1
    when lower(label) like '%' || lower('levaquin') || '%' then 1
    when lower(label) like '%' || lower('levofloxacin') || '%' then 1
    when lower(label) like '%' || lower('lincocin') || '%' then 1
    when lower(label) like '%' || lower('macrobid') || '%' then 1
    when lower(label) like '%' || lower('macrodantin') || '%' then 1
    when lower(label) like '%' || lower('maxipime') || '%' then 1
    when lower(label) like '%' || lower('mefoxin') || '%' then 1
    when lower(label) like '%' || lower('metronidazole') || '%' then 1
    when lower(label) like '%' || lower('minocin') || '%' then 1
    when lower(label) like '%' || lower('minocycline') || '%' then 1
    when lower(label) like '%' || lower('monodox') || '%' then 1
    when lower(label) like '%' || lower('monurol') || '%' then 1
    when lower(label) like '%' || lower('morgidox') || '%' then 1
    when lower(label) like '%' || lower('moxatag') || '%' then 1
    when lower(label) like '%' || lower('moxifloxacin') || '%' then 1
    when lower(label) like '%' || lower('myrac') || '%' then 1
    when lower(label) like '%' || lower('nafcillin sodium') || '%' then 1
    when lower(label) like '%' || lower('nicazel doxy 30') || '%' then 1
    when lower(label) like '%' || lower('nitrofurantoin') || '%' then 1
    when lower(label) like '%' || lower('noroxin') || '%' then 1
    when lower(label) like '%' || lower('ocudox') || '%' then 1
    when lower(label) like '%' || lower('ofloxacin') || '%' then 1
    when lower(label) like '%' || lower('omnicef') || '%' then 1
    when lower(label) like '%' || lower('oracea') || '%' then 1
    when lower(label) like '%' || lower('oraxyl') || '%' then 1
    when lower(label) like '%' || lower('oxacillin') || '%' then 1
    when lower(label) like '%' || lower('pc pen vk') || '%' then 1
    when lower(label) like '%' || lower('pce dispertab') || '%' then 1
    when lower(label) like '%' || lower('panixine') || '%' then 1
    when lower(label) like '%' || lower('pediazole') || '%' then 1
    when lower(label) like '%' || lower('penicillin') || '%' then 1
    when lower(label) like '%' || lower('periostat') || '%' then 1
    when lower(label) like '%' || lower('pfizerpen') || '%' then 1
    when lower(label) like '%' || lower('piperacillin') || '%' then 1
    when lower(label) like '%' || lower('tazobactam') || '%' then 1
    when lower(label) like '%' || lower('primsol') || '%' then 1
    when lower(label) like '%' || lower('proquin') || '%' then 1
    when lower(label) like '%' || lower('raniclor') || '%' then 1
    when lower(label) like '%' || lower('rifadin') || '%' then 1
    when lower(label) like '%' || lower('rifampin') || '%' then 1
    when lower(label) like '%' || lower('rocephin') || '%' then 1
    when lower(label) like '%' || lower('smz-tmp') || '%' then 1
    when lower(label) like '%' || lower('septra') || '%' then 1
    when lower(label) like '%' || lower('septra ds') || '%' then 1
    when lower(label) like '%' || lower('septra') || '%' then 1
    when lower(label) like '%' || lower('solodyn') || '%' then 1
    when lower(label) like '%' || lower('spectracef') || '%' then 1
    when lower(label) like '%' || lower('streptomycin sulfate') || '%' then 1
    when lower(label) like '%' || lower('sulfadiazine') || '%' then 1
    when lower(label) like '%' || lower('sulfamethoxazole') || '%' then 1
    when lower(label) like '%' || lower('trimethoprim') || '%' then 1
    when lower(label) like '%' || lower('sulfatrim') || '%' then 1
    when lower(label) like '%' || lower('sulfisoxazole') || '%' then 1
    when lower(label) like '%' || lower('suprax') || '%' then 1
    when lower(label) like '%' || lower('synercid') || '%' then 1
    when lower(label) like '%' || lower('tazicef') || '%' then 1
    when lower(label) like '%' || lower('tetracycline') || '%' then 1
    when lower(label) like '%' || lower('timentin') || '%' then 1
    when lower(label) like '%' || lower('tobi') || '%' then 1
    when lower(label) like '%' || lower('tobramycin') || '%' then 1
    when lower(label) like '%' || lower('trimethoprim') || '%' then 1
    when lower(label) like '%' || lower('unasyn') || '%' then 1
    when lower(label) like '%' || lower('vancocin') || '%' then 1
    when lower(label) like '%' || lower('vancomycin') || '%' then 1
    when lower(label) like '%' || lower('vantin') || '%' then 1
    when lower(label) like '%' || lower('vibativ') || '%' then 1
    when lower(label) like '%' || lower('vibra-tabs') || '%' then 1
    when lower(label) like '%' || lower('vibramycin') || '%' then 1
    when lower(label) like '%' || lower('zinacef') || '%' then 1
    when lower(label) like '%' || lower('zithromax') || '%' then 1
    when lower(label) like '%' || lower('zmax') || '%' then 1
    when lower(label) like '%' || lower('zosyn') || '%' then 1
    when lower(label) like '%' || lower('zyvox') || '%' then 1
  else 0
  end as antibiotic
from mimiciii.d_items di
where linksto = 'inputevents_cv' 
or linksto = 'inputevents_mv'
)
"""

query_abtbl = \
"""
, mv as 
(
select icustay_id
, label as first_antibiotic_name
, starttime as first_antibiotic_time
, ROW_NUMBER() over (partition by icustay_id order by starttime, endtime) as rn
from inputevents_mv mv
inner join ab
    on mv.itemid = ab.itemid
    and ab.antibiotic = 1
)
, cv as 
(
select icustay_id
, label as first_antibiotic_name
, charttime as first_antibiotic_time
, ROW_NUMBER() over (partition by icustay_id order by charttime) as rn
from inputevents_cv cv
inner join ab
    on cv.itemid = ab.itemid
    and ab.antibiotic = 1
)
, ab_tbl as
(
select
    ie.subject_id, ie.hadm_id, ie.icustay_id
    , coalesce(cv.first_antibiotic_name, mv.first_antibiotic_name) as first_antibiotic_name
    , coalesce(cv.first_antibiotic_time, mv.first_antibiotic_time) as first_antibiotic_time
from icustays ie
left join mv
    on ie.icustay_id = mv.icustay_id
    and mv.first_antibiotic_time between ie.intime and ie.intime + interval '24' hour
    and mv.rn = 1
left join cv
    on ie.icustay_id = cv.icustay_id
    and cv.first_antibiotic_time between ie.intime and ie.intime + interval '24' hour
    and cv.rn = 1
)
"""

query = query_ab + query_abtbl + 'select * from ab_tbl'

ab = pd.read_sql_query(query,con)
ab.describe(include='all')

# Time of blood culture

We can aggregate the microbiology table into positive/negative cultures as follows:

```sql
select hadm_id
  , chartdate, charttime
  , SPEC_TYPE_DESC
  , max(case when org_name is not null then 1 else 0 end) as PositiveCulture
from mimiciii.microbiologyevents
group by hadm_id, chartdate, charttime, spec_type_desc
```

# Defining suspected infection

Suspected infection is defined as:

* Antibiotics within 72 hours of a culture
* A culture within 24 hours of antibiotics

So the next step is to left join to this table from the antibiotic table with the following rules:

* If first_antibiotic_time is null, do not join
* If charttime is null, use chartdate and add an extra day to allow for fuzziness
* If charttime is not null, use charttime

Here's the first stab:

In [ ]:
query = query_ab + query_abtbl + \
"""
, me as
(
select hadm_id
  , chartdate, charttime
  , spec_type_desc
  , max(case when org_name is not null then 1 else 0 end) as PositiveCulture
from mimiciii.microbiologyevents
group by hadm_id, chartdate, charttime, spec_type_desc
)
select 
  ab_tbl.*
  , me72.spec_type_desc as last72_specimen
  , coalesce(me72.charttime, me72.chartdate) as last72_time
  , me72.PositiveCulture as last72_positive
  , me24.spec_type_desc as next24_specimen
  , coalesce(me24.charttime, me24.chartdate)as next24_time
  , me24.PositiveCulture as next24_positive
from ab_tbl
-- blood culture in last 72 hours
left join me me72
  on ab_tbl.hadm_id = me72.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and 
  ( 
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me72.charttime 
    and ab_tbl.first_antibiotic_time <= me72.charttime + interval '72' hour
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me72.charttime is null
    and ab_tbl.first_antibiotic_time > me72.chartdate 
    and ab_tbl.first_antibiotic_time < me72.chartdate + interval '96' hour -- could equally do this with a date_trunc, but that's less portable
    )
  )
-- blood culture in subsequent 24 hours
left join me me24
  on ab_tbl.hadm_id = me24.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and me24.charttime is not null
  and
  (
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me24.charttime - interval '24' hour
    and ab_tbl.first_antibiotic_time <= me24.charttime 
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me24.charttime is null
    and ab_tbl.first_antibiotic_time > me24.chartdate
    and ab_tbl.first_antibiotic_time <= me24.chartdate + interval '24' hour
    )
  )
"""

ab = pd.read_sql_query(query,con)
ab.head()

For now, let's just define it based of the antibiotic dose, and look for cultures before/after. We may want to revise this definition later, looking for the first culture.

To do this, we will group the above query as:

* If any of the cultures are positive, treat that as a positive case of suspected infection

We'll group and count how often charttime is available, and how often chartdate is available. We're a bit worried about quantization due to chartdate.

In [ ]:
query = query_ab + query_abtbl + \
"""
, me as
(
select hadm_id
  , chartdate, charttime
  , spec_type_desc
  , max(case when org_name is not null then 1 else 0 end) as PositiveCulture
from mimiciii.microbiologyevents
group by hadm_id, chartdate, charttime, spec_type_desc
)
, ab_fnl as
(
select 
  ab_tbl.icustay_id
  , ab_tbl.first_antibiotic_name
  , ab_tbl.first_antibiotic_time
  , min(me72.charttime) as last72_charttime
  , min(me72.chartdate) as last72_chartdate
  , min(me24.charttime) as next24_charttime
  , min(me24.chartdate) as next24_chartdate
from ab_tbl
-- blood culture in last 72 hours
left join me me72
  on ab_tbl.hadm_id = me72.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and 
  ( 
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me72.charttime 
    and ab_tbl.first_antibiotic_time <= me72.charttime + interval '72' hour
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me72.charttime is null
    and ab_tbl.first_antibiotic_time > me72.chartdate 
    and ab_tbl.first_antibiotic_time < me72.chartdate + interval '96' hour -- could equally do this with a date_trunc, but that's less portable
    )
  )
-- blood culture in subsequent 24 hours
left join me me24
  on ab_tbl.hadm_id = me24.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and me24.charttime is not null
  and
  (
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me24.charttime - interval '24' hour
    and ab_tbl.first_antibiotic_time <= me24.charttime 
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me24.charttime is null
    and ab_tbl.first_antibiotic_time > me24.chartdate
    and ab_tbl.first_antibiotic_time <= me24.chartdate + interval '24' hour
    )
  )
group by ab_tbl.icustay_id, ab_tbl.first_antibiotic_name, ab_tbl.first_antibiotic_time
)
select
  count(icustay_id) as NumICU
  , count(first_antibiotic_name) as HasAB
  , count(last72_charttime) as last72_charttime
  , count(last72_chartdate) as last72_chartdate
  , count(case when last72_charttime is null then last72_chartdate end) as last72_chartdate_only
  , count(next24_charttime) as next24_charttime
  , count(next24_chartdate) as next24_chartdate
  , count(case when next24_charttime is null then next24_chartdate end) as next24_chartdate_only
from ab_fnl
"""

ab = pd.read_sql_query(query,con)
ab.head()

This tells us something: if the culture was drawn in the ICU (as it would be for next 24 hours), then it always has a charttime. But if it's drawn before the ICU, then it is (very infrequently) missing a charttime. Lets look at these chartdates.

In [ ]:
query = query_ab + query_abtbl + \
"""
, me as
(
select hadm_id
  , chartdate, charttime
  , spec_type_desc
  , max(case when org_name is not null then 1 else 0 end) as PositiveCulture
from mimiciii.microbiologyevents
group by hadm_id, chartdate, charttime, spec_type_desc
)
, ab_fnl as
(
select 
  ab_tbl.icustay_id
  , ab_tbl.first_antibiotic_name
  , ab_tbl.first_antibiotic_time
  , min(me72.charttime) as last72_charttime
  , min(me72.chartdate) as last72_chartdate
  , min(me24.charttime) as next24_charttime
  , min(me24.chartdate) as next24_chartdate
from ab_tbl
-- blood culture in last 72 hours
left join me me72
  on ab_tbl.hadm_id = me72.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and 
  ( 
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me72.charttime 
    and ab_tbl.first_antibiotic_time <= me72.charttime + interval '72' hour
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me72.charttime is null
    and ab_tbl.first_antibiotic_time > me72.chartdate 
    and ab_tbl.first_antibiotic_time < me72.chartdate + interval '96' hour -- could equally do this with a date_trunc, but that's less portable
    )
  )
-- blood culture in subsequent 24 hours
left join me me24
  on ab_tbl.hadm_id = me24.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and me24.charttime is not null
  and
  (
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me24.charttime - interval '24' hour
    and ab_tbl.first_antibiotic_time <= me24.charttime 
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me24.charttime is null
    and ab_tbl.first_antibiotic_time > me24.chartdate
    and ab_tbl.first_antibiotic_time <= me24.chartdate + interval '24' hour
    )
  )
group by ab_tbl.icustay_id, ab_tbl.first_antibiotic_name, ab_tbl.first_antibiotic_time
)
select
  ab_fnl.*
from ab_fnl
where last72_charttime is null and last72_chartdate is not null;
"""

ab = pd.read_sql_query(query,con)
ab.head(n=10)

For these cases, let's assume the first antibiotic time is the starting time of suspicion of infection.

In [ ]:
query = query_ab + query_abtbl + \
"""
, me as
(
select hadm_id
  , chartdate, charttime
  , spec_type_desc
  , max(case when org_name is not null then 1 else 0 end) as PositiveCulture
from mimiciii.microbiologyevents
group by hadm_id, chartdate, charttime, spec_type_desc
)
, ab_fnl as
(
select 
  ab_tbl.icustay_id
  , ab_tbl.first_antibiotic_name
  , ab_tbl.first_antibiotic_time
  , me72.charttime as last72_charttime
  , me72.chartdate as last72_chartdate
  , me24.charttime as next24_charttime
  , me24.chartdate as next24_chartdate
  
  , me72.positiveculture as last72_positiveculture
  , me72.spec_type_desc as last72_specimen
  , me24.positiveculture as next24_positiveculture
  , me24.spec_type_desc as next24_specimen
  
  , ROW_NUMBER() over (partition by ab_tbl.icustay_id order by coalesce(me72.charttime, me24.charttime, me72.chartdate))
      as rn
from ab_tbl
-- blood culture in last 72 hours
left join me me72
  on ab_tbl.hadm_id = me72.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and 
  ( 
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me72.charttime 
    and ab_tbl.first_antibiotic_time <= me72.charttime + interval '72' hour
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me72.charttime is null
    and ab_tbl.first_antibiotic_time > me72.chartdate 
    and ab_tbl.first_antibiotic_time < me72.chartdate + interval '96' hour -- could equally do this with a date_trunc, but that's less portable
    )
  )
-- blood culture in subsequent 24 hours
left join me me24
  on ab_tbl.hadm_id = me24.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and me24.charttime is not null
  and
  (
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me24.charttime - interval '24' hour
    and ab_tbl.first_antibiotic_time <= me24.charttime 
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me24.charttime is null
    and ab_tbl.first_antibiotic_time > me24.chartdate
    and ab_tbl.first_antibiotic_time <= me24.chartdate + interval '24' hour
    )
  )
)
select
  ab_fnl.icustay_id
  -- time of suspected infection: either the culture time (if before antibiotic), or the antibiotic time
  , case 
      when last72_charttime is not null
        then last72_charttime
      when next24_charttime is not null or last72_chartdate is not null
        then first_antibiotic_time
    else null
  end as suspected_infection_time
  -- the specimen that was cultured
  , case 
      when last72_charttime is not null or last72_chartdate is not null
        then last72_specimen
      when next24_charttime is not null 
        then next24_specimen
    else null
  end as specimen
  -- whether the cultured specimen ended up being positive or not
  , case 
      when last72_charttime is not null or last72_chartdate is not null
        then last72_positiveculture
      when next24_charttime is not null 
        then next24_positiveculture
    else null
  end as positiveculture
from ab_fnl
where rn = 1
order by icustay_id;
"""

ab = pd.read_sql_query(query,con)
ab.head(n=10)

The above is our final definition of suspected infection.